# LPI Regression: GroupKFold Cross-Validation with PCA + Multiple Models
### Liquefaction Potential Index Prediction from Borehole Geotechnical Features

**Dataset:** `combined_6.5_7_7.5 (1).csv`  
**Target:** `lpi` — Liquefaction Potential Index (continuous regression target, range 0–61)  
**Strategy:** `StandardScaler → PCA (95% variance) → GroupKFold(k=10, group=BoreholeID) → 8 Regression Models`  

---
**Notebook Structure:**
0. Imports & Configuration  
1. Data Loading & Exploratory Data Analysis (EDA)  
2. Preprocessing & Feature Engineering  
3. PCA Analysis (Dimensionality Reduction)  
4. GroupKFold Cross-Validation Setup  
5. Model Definition (8 Regressors)  
6. GroupKFold Training & Evaluation  
7. Model Comparison Visualizations  
8. Best Model — OOF Residual Analysis  
9. PCA Component Importance  
10. Original Feature Importance  
11. Spatial Distribution of Predictions  
12. Statistical Significance Testing (Friedman Test)  
13. Results Summary & Export  

## Section 0: Imports & Configuration

In [ ]:
import os
import warnings
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GroupKFold, cross_validate
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

try:
    from xgboost import XGBRegressor
    HAS_XGB = True
except ImportError:
    HAS_XGB = False
    print('XGBoost not found — skipping')

warnings.filterwarnings('ignore')
np.random.seed(42)

# ─── CONFIGURATION ────────────────────────────────────────────
CSV_PATH  = 'combined_6.5_7_7.5.csv' if os.path.exists('combined_6.5_7_7.5.csv') else 'combined_6.5_7_7.5 (1).csv'
OUT       = 'results'
N_FOLDS   = 10
PCA_VAR   = 0.95   # retain 95% of variance
SEED      = 42
os.makedirs(OUT, exist_ok=True)

plt.rcParams.update({'figure.dpi': 120, 'font.size': 11})
print('Setup complete.')

## Section 1: Data Loading & Exploratory Data Analysis

In [ ]:
df_raw = pd.read_csv(CSV_PATH)
print(f'Raw shape: {df_raw.shape}')

META_COLS    = ['Latitude', 'Longitude', 'BoreholeID']
TARGET_COL   = 'lpi'
LPI_COMP     = [c for c in df_raw.columns if c.startswith('lpi_component')]
FEATURE_COLS = [c for c in df_raw.columns if c not in META_COLS + [TARGET_COL] + LPI_COMP]

print(f'\nFeatures ({len(FEATURE_COLS)}): {FEATURE_COLS}')
print(f'LPI components excluded ({len(LPI_COMP)}): {LPI_COMP}')
print(f'Target: {TARGET_COL}')
print(f'\nTarget (LPI) statistics:')
display(df_raw[TARGET_COL].describe().to_frame().T)

In [ ]:
# Missing values & duplicates
print('Missing values per column:')
print(df_raw[FEATURE_COLS + [TARGET_COL]].isnull().sum().to_string())

n_dup = df_raw.duplicated(subset=FEATURE_COLS).sum()
print(f'\nDuplicate rows (by features): {n_dup}')
df = df_raw.drop_duplicates(subset=FEATURE_COLS).copy()
print(f'Shape after deduplication: {df.shape}')

In [ ]:
# EDA Plots
fig, axes = plt.subplots(1, 3, figsize=(16, 4))

# Distribution
axes[0].hist(df[TARGET_COL], bins=40, color='steelblue', edgecolor='white')
axes[0].axvline(df[TARGET_COL].mean(), color='red', linestyle='--',
                label=f"Mean={df[TARGET_COL].mean():.2f}")
axes[0].set(title='LPI Distribution', xlabel='LPI', ylabel='Count')
axes[0].legend()

# Q-Q plot
(osm, osr), (s, i, _) = stats.probplot(df[TARGET_COL], dist='norm')
axes[1].scatter(osm, osr, s=8, alpha=0.5)
axes[1].plot(osm, s*np.array(osm)+i, 'r-')
axes[1].set(title='Q-Q Plot of LPI', xlabel='Theoretical Q', ylabel='Sample Q')

# Boxplot by earthquake magnitude
mag_vals = sorted(df['M_3'].unique())
axes[2].boxplot([df[df['M_3']==m][TARGET_COL].values for m in mag_vals],
                labels=[str(m) for m in mag_vals], patch_artist=True)
axes[2].set(title='LPI by Earthquake Magnitude (M_3)', xlabel='Mw', ylabel='LPI')

plt.suptitle('Figure 1 — Target Variable: Exploratory Analysis', fontweight='bold')
plt.tight_layout()
plt.savefig(f'{OUT}/01_eda_overview.png', bbox_inches='tight')
plt.show()

In [ ]:
# Correlation heatmap
corr = df[FEATURE_COLS + [TARGET_COL]].corr()
mask = np.triu(np.ones_like(corr, dtype=bool), k=1)
fig, ax = plt.subplots(figsize=(14, 11))
sns.heatmap(corr, mask=mask, cmap='RdBu_r', center=0, vmin=-1, vmax=1,
            annot=False, linewidths=0.3, ax=ax)
ax.set_title('Figure 2 — Feature Correlation Matrix (incl. LPI Target)', fontweight='bold')
plt.tight_layout()
plt.savefig(f'{OUT}/02_correlation_heatmap.png', bbox_inches='tight')
plt.show()

## Section 2: Preprocessing

In [ ]:
X   = df[FEATURE_COLS].values
y   = df[TARGET_COL].values
lat = df['Latitude'].values
lon = df['Longitude'].values
groups = df['BoreholeID'].values

print(f'Feature matrix X: {X.shape}')
print(f'Target vector  y: {y.shape}')
print(f'LPI range: [{y.min():.3f}, {y.max():.3f}]')

variances = pd.Series(X.var(axis=0), index=FEATURE_COLS).sort_values(ascending=False)
print('\nTop 10 features by raw variance:')
display(variances.head(10).to_frame('Variance'))

## Section 3: PCA Analysis

In [ ]:
scaler_full = StandardScaler()
X_scaled    = scaler_full.fit_transform(X)
pca_full    = PCA(random_state=SEED).fit(X_scaled)

explained  = pca_full.explained_variance_ratio_
cumulative = np.cumsum(explained)

n85 = np.argmax(cumulative >= 0.85) + 1
n90 = np.argmax(cumulative >= 0.90) + 1
n95 = np.argmax(cumulative >= 0.95) + 1
N_PCA = n95

print(f'Components for 85% variance: {n85}')
print(f'Components for 90% variance: {n90}')
print(f'Components for 95% variance: {n95}  << SELECTED')

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(13, 5))

axes[0].bar(range(1, len(explained)+1), explained*100, color='steelblue', alpha=0.7)
axes[0].plot(range(1, len(explained)+1), explained*100, 'ro-', ms=4)
axes[0].set(title='Scree Plot', xlabel='PC', ylabel='Var. Explained (%)')
axes[0].set_xlim(0.5, min(24, len(explained))+0.5)

axes[1].plot(range(1, len(cumulative)+1), cumulative*100, 'b-o', ms=4)
for thresh, col, lbl in [(85,'orange','85%'), (90,'red','90%'), (95,'green','95%')]:
    axes[1].axhline(thresh, color=col, linestyle='--', label=lbl)
axes[1].axvline(n95, color='green', linestyle=':', alpha=0.7)
axes[1].set(title='Cumulative Explained Variance', xlabel='N Components', ylabel='Cumulative (%)')
axes[1].legend()
axes[1].set_xlim(0.5, 24.5)

plt.suptitle(f'Figure 3 — PCA: {N_PCA} components retain 95% variance', fontweight='bold')
plt.tight_layout()
plt.savefig(f'{OUT}/03_pca_variance.png', bbox_inches='tight')
plt.show()

In [ ]:
pca_viz = PCA(n_components=N_PCA, random_state=SEED).fit(X_scaled)
loadings = pd.DataFrame(
    pca_viz.components_.T,
    index=FEATURE_COLS,
    columns=[f'PC{i+1}' for i in range(N_PCA)]
)

fig, ax = plt.subplots(figsize=(max(10, N_PCA), 8))
kws = {'annot': True, 'fmt': '.2f'} if N_PCA <= 10 else {'annot': False}
sns.heatmap(loadings, cmap='RdBu_r', center=0, vmin=-1, vmax=1, ax=ax, linewidths=0.3, **kws)
ax.set_title(f'Figure 4 — PCA Loadings (Top {N_PCA} Components)', fontweight='bold')
plt.tight_layout()
plt.savefig(f'{OUT}/04_pca_loadings.png', bbox_inches='tight')
plt.show()

## Section 4: GroupKFold Cross-Validation Setup

In [ ]:
kf = GroupKFold(n_splits=N_FOLDS)

def make_pipe(model):
    """StandardScaler -> PCA (variance=PCA_VAR) -> Regressor pipeline."""
    return Pipeline([
        ('scaler', StandardScaler()),
        ('pca',    PCA(n_components=PCA_VAR, svd_solver='full', random_state=SEED)),
        ('model',  model)
    ])

print(f'GroupKFold: k={N_FOLDS}, group=BoreholeID')
print(f'Pipeline: StandardScaler -> PCA(var={PCA_VAR}) -> Regressor')

## Section 5: Model Definitions (8 Regressors)

In [ ]:
MODELS = {
    'Linear Regression'  : make_pipe(LinearRegression()),
    'Ridge (a=1.0)'      : make_pipe(Ridge(alpha=1.0, random_state=SEED)),
    'Lasso (a=0.1)'      : make_pipe(Lasso(alpha=0.1, random_state=SEED, max_iter=5000)),
    'ElasticNet'         : make_pipe(ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=SEED, max_iter=5000)),
    'Random Forest'      : make_pipe(RandomForestRegressor(n_estimators=200, random_state=SEED, n_jobs=-1)),
    'Gradient Boosting'  : make_pipe(GradientBoostingRegressor(n_estimators=200, learning_rate=0.05,
                                                                max_depth=4, random_state=SEED)),
    'SVR (RBF)'          : make_pipe(SVR(kernel='rbf', C=10.0, epsilon=0.5, gamma='scale')),
}

if HAS_XGB:
    MODELS['XGBoost'] = make_pipe(XGBRegressor(n_estimators=200, learning_rate=0.05,
                                               max_depth=5, random_state=SEED,
                                               n_jobs=-1, verbosity=0))

print(f'Models ({len(MODELS)}):')
for name in MODELS:
    print(f'  - {name}')

## Section 6: GroupKFold Training & Evaluation

In [ ]:
SCORING = {'r2': 'r2', 'neg_mse': 'neg_mean_squared_error', 'neg_mae': 'neg_mean_absolute_error'}
cv_results = {}

for name, pipe in MODELS.items():
    print(f'Training {name}...', end=' ', flush=True)
    cv = cross_validate(pipe, X, y, groups=groups, cv=kf, scoring=SCORING,
                        return_train_score=True, n_jobs=-1)
    
    r2_cv   = cv['test_r2']
    mse_cv  = -cv['test_neg_mse']
    mae_cv  = -cv['test_neg_mae']
    rmse_cv = np.sqrt(mse_cv)
    
    cv_results[name] = {
        'R2': r2_cv, 'RMSE': rmse_cv, 'MAE': mae_cv,
        'train_R2': cv['train_r2']
    }
    print(f'R2={r2_cv.mean():.4f}+-{r2_cv.std():.4f}  RMSE={rmse_cv.mean():.3f}+-{rmse_cv.std():.3f}')

In [ ]:
# Summary Table
summary = pd.DataFrame({
    name: {
        'R2 mean': f"{v['R2'].mean():.4f}",
        'R2 std':  f"{v['R2'].std():.4f}",
        'RMSE mean': f"{v['RMSE'].mean():.3f}",
        'RMSE std':  f"{v['RMSE'].std():.3f}",
        'MAE mean': f"{v['MAE'].mean():.3f}",
        'MAE std':  f"{v['MAE'].std():.3f}",
        'Train R2': f"{v['train_R2'].mean():.4f}",
    }
    for name, v in cv_results.items()
}).T

print('=== 10-Fold Cross-Validation Results (GroupKFold + PCA + Model) ===')
display(summary)

## Section 7: Model Comparison Visualizations

In [ ]:
names      = list(cv_results.keys())
r2_means   = [cv_results[n]['R2'].mean()   for n in names]
r2_stds    = [cv_results[n]['R2'].std()    for n in names]
rmse_means = [cv_results[n]['RMSE'].mean() for n in names]
rmse_stds  = [cv_results[n]['RMSE'].std()  for n in names]
mae_means  = [cv_results[n]['MAE'].mean()  for n in names]
mae_stds   = [cv_results[n]['MAE'].std()   for n in names]
train_r2s  = [cv_results[n]['train_R2'].mean() for n in names]

colors = plt.cm.get_cmap('tab10', len(names)).colors

fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for ax, means, stds, ylabel, title, best_high in [
    (axes[0], r2_means,   r2_stds,   'R2',   'R2 Score (10-Fold CV)',  True),
    (axes[1], rmse_means, rmse_stds, 'RMSE', 'RMSE (10-Fold CV)',      False),
    (axes[2], mae_means,  mae_stds,  'MAE',  'MAE (10-Fold CV)',       False),
]:
    bars = ax.bar(range(len(names)), means, yerr=stds, capsize=5,
                  color=colors, alpha=0.85, edgecolor='k', linewidth=0.5)
    best = np.argmax(means) if best_high else np.argmin(means)
    bars[best].set_edgecolor('gold'); bars[best].set_linewidth(3)
    ax.set_xticks(range(len(names)))
    ax.set_xticklabels(names, rotation=35, ha='right', fontsize=9)
    ax.set(ylabel=ylabel, title=title)

plt.suptitle('Figure 5 — Model Comparison: GroupKFold(PCA+Model)', fontweight='bold')
plt.tight_layout()
plt.savefig(f'{OUT}/05_model_comparison.png', bbox_inches='tight')
plt.show()

In [ ]:
# Box plots of per-fold R2
fig, ax = plt.subplots(figsize=(12, 5))
bp = ax.boxplot([cv_results[n]['R2'] for n in names], patch_artist=True, labels=names,
                medianprops=dict(color='red', linewidth=2))
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color); patch.set_alpha(0.7)
ax.set_xticklabels(names, rotation=35, ha='right', fontsize=9)
ax.set(title='Figure 6 — Per-Fold R2 Distribution (10-Fold CV)', ylabel='R2')
plt.tight_layout()
plt.savefig(f'{OUT}/06_fold_r2_boxplots.png', bbox_inches='tight')
plt.show()

In [ ]:
# Train vs Test R2
x, w = np.arange(len(names)), 0.35
fig, ax = plt.subplots(figsize=(12, 5))
ax.bar(x-w/2, train_r2s, w, label='Train R2', color='steelblue', alpha=0.8, edgecolor='k')
ax.bar(x+w/2, r2_means,  w, label='Test R2',  color='coral',     alpha=0.8, edgecolor='k')
ax.set_xticks(x)
ax.set_xticklabels(names, rotation=35, ha='right', fontsize=9)
ax.set(title='Figure 7 — Train vs Test R2 (Overfitting Check)', ylabel='R2')
ax.legend()
plt.tight_layout()
plt.savefig(f'{OUT}/07_train_vs_test_r2.png', bbox_inches='tight')
plt.show()

## Section 8: Best Model — OOF Residual Analysis

In [ ]:
best_name = names[np.argmax(r2_means)]
print(f'Best model: {best_name}  (mean CV R2 = {max(r2_means):.4f})')

best_pipe = MODELS[best_name]
oof_pred  = np.zeros(len(y))
oof_true  = np.zeros(len(y))

for _, (tr, va) in enumerate(kf.split(X, y, groups)):
    best_pipe.fit(X[tr], y[tr])
    oof_pred[va] = best_pipe.predict(X[va])
    oof_true[va] = y[va]

residuals  = oof_true - oof_pred
final_r2   = r2_score(oof_true, oof_pred)
final_rmse = np.sqrt(mean_squared_error(oof_true, oof_pred))
final_mae  = mean_absolute_error(oof_true, oof_pred)

print(f'OOF R2   = {final_r2:.4f}')
print(f'OOF RMSE = {final_rmse:.3f}')
print(f'OOF MAE  = {final_mae:.3f}')

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(13, 10))

axes[0,0].scatter(oof_true, oof_pred, s=15, alpha=0.4)
lo, hi = oof_true.min(), oof_true.max()
axes[0,0].plot([lo,hi],[lo,hi],'r--',lw=2, label='Perfect fit')
axes[0,0].set(title=f'Actual vs Predicted — {best_name}\nR2={final_r2:.4f}',
              xlabel='Actual LPI', ylabel='Predicted LPI')
axes[0,0].legend()

axes[0,1].scatter(oof_pred, residuals, s=15, alpha=0.4, color='coral')
axes[0,1].axhline(0, color='k', linestyle='--')
axes[0,1].set(title='Residuals vs Predicted', xlabel='Predicted LPI', ylabel='Residuals')

axes[1,0].hist(residuals, bins=40, color='mediumseagreen', edgecolor='white')
axes[1,0].axvline(0, color='red', linestyle='--')
axes[1,0].set(title='Residual Distribution', xlabel='Residual', ylabel='Count')

(osm2, osr2), (s2, i2, _) = stats.probplot(residuals, dist='norm')
axes[1,1].scatter(osm2, osr2, s=10, alpha=0.5)
axes[1,1].plot(osm2, s2*np.array(osm2)+i2, 'r-')
axes[1,1].set(title='Q-Q Plot of Residuals', xlabel='Theoretical Q', ylabel='Residual Q')

plt.suptitle(f'Figure 8 — Best Model Analysis: {best_name}', fontweight='bold')
plt.tight_layout()
plt.savefig(f'{OUT}/08_best_model_residuals.png', bbox_inches='tight')
plt.show()

## Section 9: PCA Component Importance

In [ ]:
pca_final = PCA(n_components=N_PCA, random_state=SEED)
X_pca     = pca_final.fit_transform(StandardScaler().fit_transform(X))

pc_corr = pd.DataFrame([
    {'PC': f'PC{i+1}',
     'Pearson_r': stats.pearsonr(X_pca[:,i], y)[0],
     'p_value':   stats.pearsonr(X_pca[:,i], y)[1],
     'Var_explained_%': pca_final.explained_variance_ratio_[i]*100}
    for i in range(N_PCA)
])

display(pc_corr)
pc_corr.to_csv(f'{OUT}/pca_component_correlations.csv', index=False)

fig, axes = plt.subplots(1, 2, figsize=(13, 5))
colors_bar = ['coral' if r < 0 else 'steelblue' for r in pc_corr['Pearson_r']]
axes[0].bar(pc_corr['PC'], pc_corr['Pearson_r'], color=colors_bar, edgecolor='k')
axes[0].axhline(0, color='k', lw=1)
axes[0].set(title='PC Correlation with LPI', xlabel='PC', ylabel='Pearson r')
axes[1].bar(pc_corr['PC'], pc_corr['Var_explained_%'], color='mediumseagreen', edgecolor='k')
axes[1].set(title='Variance Explained per PC', xlabel='PC', ylabel='Var. Explained (%)')
plt.suptitle('Figure 9 — PCA Component Analysis', fontweight='bold')
plt.tight_layout()
plt.savefig(f'{OUT}/09_pca_importance.png', bbox_inches='tight')
plt.show()

## Section 10: Original Feature Importance (via PCA Loadings)

In [ ]:
var_exp  = pca_final.explained_variance_ratio_
feat_imp = np.sum(np.abs(pca_final.components_) * var_exp[:, None], axis=0)
feat_df  = pd.DataFrame({'Feature': FEATURE_COLS, 'Importance': feat_imp})
feat_df  = feat_df.sort_values('Importance', ascending=False)

display(feat_df)
feat_df.to_csv(f'{OUT}/feature_importance_pca.csv', index=False)

fig, ax = plt.subplots(figsize=(10, 6))
cmap = plt.cm.get_cmap('viridis', len(feat_df))
ax.barh(feat_df['Feature'][::-1], feat_df['Importance'][::-1],
        color=[cmap(i) for i in range(len(feat_df))][::-1], edgecolor='k', linewidth=0.3)
ax.set(title='Figure 10 — Feature Importance (PCA Loadings x Variance)',
       xlabel='PCA-Weighted Importance Score')
plt.tight_layout()
plt.savefig(f'{OUT}/10_feature_importance.png', bbox_inches='tight')
plt.show()

## Section 11: Spatial Distribution of Predictions

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for ax, data, title, cmap in [
    (axes[0], oof_true,  'Actual LPI',                  'RdYlGn_r'),
    (axes[1], oof_pred,  f'Predicted LPI ({best_name})', 'RdYlGn_r'),
    (axes[2], residuals, 'Residuals',                    'RdBu'),
]:
    sc = ax.scatter(lon, lat, c=data, cmap=cmap, s=20, alpha=0.7)
    plt.colorbar(sc, ax=ax)
    ax.set(title=title, xlabel='Longitude', ylabel='Latitude')

plt.suptitle('Figure 11 — Spatial Distribution of LPI Predictions', fontweight='bold')
plt.tight_layout()
plt.savefig(f'{OUT}/11_spatial_lpi_map.png', bbox_inches='tight')
plt.show()

spatial_df = pd.DataFrame({'Latitude': lat, 'Longitude': lon,
                            'LPI_actual': oof_true, 'LPI_predicted': oof_pred,
                            'Residual': residuals})
spatial_df.to_csv(f'{OUT}/spatial_predictions.csv', index=False)
print('Saved spatial_predictions.csv')

## Section 12: Statistical Significance Testing

In [ ]:
all_r2s = [cv_results[n]['R2'] for n in names]
stat, pval = stats.friedmanchisquare(*all_r2s)
print(f'Friedman Test (non-parametric):')
print(f'  chi2-statistic = {stat:.4f}')
print(f'  p-value        = {pval:.6e}')
print(f'  Significant (alpha=0.05): {pval < 0.05}')
print('')
print('Interpretation: The Friedman test evaluates whether model R2 scores')
print('differ significantly across the 10 folds. p<0.001 confirms significant')
print('differences exist between the 8 models.')

## Section 13: Results Summary & Export

In [ ]:
print('='*60)
print('FINAL RESULTS SUMMARY')
print('='*60)
print(f'Dataset           : {len(df)} samples x {len(FEATURE_COLS)} features (after dedup)')
print(f'PCA components    : {N_PCA} (retaining 95% variance from 24 features)')
print(f'CV strategy       : GroupKFold (k={N_FOLDS}, group=BoreholeID)')
print('')
print(f'{"Model":<22} {"R2 mean":>10} {"R2 std":>8} {"RMSE mean":>11} {"MAE mean":>10}')
print('-'*65)
for name in names:
    v = cv_results[name]
    print(f'{name:<22} {v["R2"].mean():>10.4f} {v["R2"].std():>8.4f} '
          f'{v["RMSE"].mean():>11.3f} {v["MAE"].mean():>10.3f}')

print('-'*65)
print(f'\nBest model: {best_name}')
print(f'OOF R2   = {final_r2:.4f}')
print(f'OOF RMSE = {final_rmse:.3f} LPI units')
print(f'OOF MAE  = {final_mae:.3f} LPI units')
print(f'\nFriedman chi2={stat:.2f}, p={pval:.2e} (models differ significantly)')

print(f'\nAll outputs saved to: {OUT}/')